**False Positive Reduction for Lung Nodule Detection:** The following code implements a Convolutional Neural Network (CNN) for false positive reduction in lung nodule detection. The input to the model is a patch of a nodule, and the target is binary classification—whether the patch contains a nodule or not.

In [1]:
import pickle

with open('Image_LUNG.pkl', 'rb') as f:
   image = pickle.load(f)

with open('label_LUNG.pkl', 'rb') as f:
   label1 = pickle.load(f)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pickle
import numpy as np

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# Initialize arrays to store image data and labels
N = len(label1['output'])
image_data = np.zeros((N, 60, 60))
label = np.zeros((N, 1))
k = 0
for i in range(N):
    try:
        # Populate image_data and label arrays
        label[k, :] = label1['output'][i]
        image_data[k, :, :] = image['output'][i]
        k += 1
    except:
        # Skip any data that cannot be processed
        continue

# Shuffle and split the data into training and validation sets
N1 = len(label)
np.random.seed(20)
ind = np.random.permutation(N1)
image_data = image_data[ind, :, :]
label = label[ind, :]

# Normalize image data and convert labels to integer type
image_data = image_data.reshape(-1, 1, 60, 60).astype('float32') / 255
label = np.squeeze(label).astype('int64')

# Define the split ratios for training and validation data
training_split = 0.9
num_images = image_data.shape[0]
train_size = int(training_split * num_images)
val_size = num_images - train_size

# Create PyTorch tensors for training and validation data
image_data1_train = torch.tensor(image_data[:train_size])
image_data1_val = torch.tensor(image_data[train_size:])
label1_train = torch.tensor(label[:train_size])
label1_val = torch.tensor(label[train_size:])

# Create datasets and data loaders for training and validation sets
train_dataset = data.TensorDataset(image_data1_train, label1_train)
val_dataset = data.TensorDataset(image_data1_val, label1_val)

train_loader = data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = data.DataLoader(val_dataset, batch_size=16, shuffle=False)

# Define the CNN model architecture
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Define the layers of the CNN
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 15 * 15, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 2)
    def forward(self, x):
        # Define the forward pass
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # Output layer
        return x

# Initialize the model and move it to the appropriate device
model = CNNModel().to(device)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)        
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the model parameters        
        running_loss += loss.item()    
    # Print loss after each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Evaluate the model
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
# Print validation accuracy
print(f'\nValidation Accuracy: {100 * correct / total:.2f}%')
# Save the trained model
torch.save(model.state_dict(), 'FPR_LUNG3.pth')



Using device: cpu
Epoch [1/5], Loss: 0.6744
Epoch [2/5], Loss: 0.5881
Epoch [3/5], Loss: 0.4960
Epoch [4/5], Loss: 0.3884
Epoch [5/5], Loss: 0.2898

Validation Accuracy: 83.71%
